# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
product_list = ["maíz", "sorgo", "trigo", "maní",
                "miel de abeja nativa", "miel de abeja apis", "poroto",
                "avena", "cebada", "papa", "haba", "arveja", "mandioca",
                "zapallo", "sandía", "melón", "quínoa",
                "hortalizas y fruta", "frutos silvestres", "insumos",
                "leña", "productos de caza \/ pesca",
                "forrajeras nativas", "forrajeras exóticas",
                "alfalfa cosechad[oa]", "otros"]
translation_list = ["corn", "sorghum", "wheat", "peanut",
                    "native_bee_honey", "apis_honey", "beans", "oats",
                    "barley", "potato", "broad_bean", "peas",
                    "cassava", "pumpkin", "watermelon", "melon",
                    "quinoa", "vegetables_and_fruit", "wild_fruits",
                    "supply", "firewood", "hunting_or_fishing_products",
                    "native_forages", "exotic_forages",
                    "harvested_alfalfa"]
# column headers not including product column
column_headers = ["has_sold", "commercial_fraction", "quantity_sold",
                  "minimum_unit_price", "maximum_unit_price"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_date_range(section_search):
    date_range = ""

    if len(section_search) == 1:
        section_string = section_search[0]

        date_regex = r"([\w ]+\d+ *[-–] *[\w ]+\d+)"
        date_search = re.findall(date_regex, section_string)

        if len(date_search) == 1:
            date_range = date_search[0]
        else:
            date_range = "UNENCOUNTERED FORMAT"
    else:
        date_range = "UNENCOUNTERED FORMAT"

    return date_range


def parse_section_5_3(section_search, product_list,
                      translation_list, column_headers):
    section_5_3_data = []

    if len(section_search) == 1:
        section_string = section_search[0]

        for i in range(len(product_list)-1):
            item_dict = {}
            item_1 = product_list[i]
            item_2 = product_list[i+1]
            term = translation_list[i]

            row_search = find_between(item_1, item_2, section_string)

            item_dict["product"] = term

            # if product row found
            if len(row_search) == 1:
                row_string = row_search[0]
                row_list = row_string.split("\n")
                row_list_len = len(row_list)

                if row_list_len >= 13:
                    # skip odd indexes (1, 3, 5, 7)
                    # drop first two rows (start count at -2)
                    count = -2
                    for j in range(row_list_len):
                        if count > 8:
                            break
                        if count >= 0 and count % 2 == 0:
                            col = column_headers[count//2]
                            entry = row_list[j]
                            entry = entry.strip()
                            item_dict[col] = entry
                        count += 1
                else:
                    for col in column_headers:
                        item_dict[col] = "UNENCOUNTERED FORMAT"
            else:
                for col in column_headers:
                    item_dict[col] = "UNENCOUNTERED FORMAT"

            section_5_3_data.append(item_dict)
    else:
        section_5_3_data = [{}]

    return section_5_3_data


def parse_otros_5_3(otros_search, column_headers):
    otros_5_3_data = []

    # parsing section for otros
    if len(otros_search) == 1:
        otros_string = otros_search[0]
        otros_list = otros_string.split("\n")
        otros_list = otros_list[1:]

        if len(otros_list) >= 43:
            # skip odd indexes (1, 3, 5, 7)
            count = 0
            count_restart = -2
            count_end = 12

            if len(otros_list) == 49 or len(otros_list) == 65:
                count_restart = -4

            item_dict = {}
            for item in otros_list:
                if count == count_end:
                    count = count_restart
                    otros_5_3_data.append(item_dict)
                    item_dict = {}
                if count == 0:
                    stripped_item = item.strip()
                    item_dict["product"] = stripped_item
                if count > 0 and count % 2 == 0:
                    col = column_headers[(count-2)//2]
                    stripped_item = item.strip()
                    item_dict[col] = stripped_item
                count += 1
        else:
            otros_5_3_data = [{}]
    else:
        otros_5_3_data = [{}]

    return otros_5_3_data

### Process the Whole File

In [ ]:
def process_files(source_directory, product_list,
                  translation_list, column_headers):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            section_start = "v-3: ventas de productos"
            section_end = "v-4: otros ingresos"
            section_search = find_between(section_start, section_end, contents)

            otros_start = r"v-3[\w\W]*otros[^\n]*\n"
            otros_search = find_between(otros_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            date_range = parse_date_range(section_search)
            section_5_3_data = parse_section_5_3(section_search, product_list,
                                                 translation_list, column_headers)
            otros_5_3_data = parse_otros_5_3(otros_search, column_headers)

            data_dict = {}

            # check to make sure files contain the right amount of entries
            if len(section_5_3_data) != 25:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of section_data: {len(section_5_3_data)}\n")

            if len(otros_5_3_data) != 3 and len(otros_5_3_data) != 4:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of otros_data: {len(otros_5_3_data)}\n")

            for row in section_5_3_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

            for row in otros_5_3_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

In [ ]:
def clean_date_range(dataf):
    replace_dict = {"agosto 2018 - agosto 2019": "august 2018-2019",
                    "septiembre 2018 - septiembre 2019": "september 2018-2019",
                    "octubre 2018 – octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre2019": "october 2018-2019",
                    "octubre 2018 – octubre  2019": "october 2018-2019"}
    dataf["date_range"] = dataf["date_range"].replace(replace_dict)

    return dataf


def clean_has_sold(dataf):
    map_dict = {"si": "yes",
                "n": "no"}

    dataf["has_sold"] = dataf["has_sold"].replace(map_dict)

    return dataf


def clean_commercial_fraction(dataf):
    map_dict = {"total": 1.0,
                "totalidad": 1.0,
                "mitad": 0.5,
                "la mitad": 0.5,
                "2/3": (2/3),
                "½": 0.5,
                "3/4": 0.75,
                "75 %": 0.75,
                "75%": 0.75,
                "50%": 0.5,
                "25%": 0.25,
                "100%": 1.0,
                "100 %": 1.0,
                "1/2": 0.5,
                "1/4": 0.25,
                "4": 1.0,
                "3": 0.75,
                "2": 0.5,
                "1": 0.25}

    dataf["commercial_fraction"] = dataf["commercial_fraction"].replace(map_dict)

    return dataf


def clean_quantity_sold(dataf):
    replace_filter_comma = dataf["quantity_sold"].str.contains(r"\.\d{3}",
                                                               na=False)
    replace_series_comma = dataf["quantity_sold"].str.replace(".", "")

    dataf["quantity_sold"] = dataf["quantity_sold"].where(~replace_filter_comma,
                                                          replace_series_comma)

    replace_filter_period = dataf["quantity_sold"].str.contains(r",\d{1,2}",
                                                                na=False)
    replace_series_period = dataf["quantity_sold"].str.replace(",", ".")

    dataf["quantity_sold"] = dataf["quantity_sold"].where(~replace_filter_period,
                                                          replace_series_period)

    # create column for the units specified in quantity sold
    replace_filter_units = r"(\d*\.?\d*) ?([\w\W]*)"
    unit_extraction = dataf["quantity_sold"].str.extract(replace_filter_units)

    dataf["quantity_sold"] = unit_extraction[0]
    dataf["unit_of_measure"] = unit_extraction[1]

    return dataf


def clean_unit_price(dataf):
    replace_filter_min = dataf["minimum_unit_price"].str.contains(r",\d{2}", na=False)
    replace_filter_max = dataf["maximum_unit_price"].str.contains(r",\d{2}", na=False)

    replace_series_min = dataf["minimum_unit_price"].str.replace(",", ".")
    replace_series_max = dataf["maximum_unit_price"].str.replace(",", ".")

    dataf["minimum_unit_price"] = dataf["minimum_unit_price"].where(~replace_filter_min, replace_series_min)
    dataf["maximum_unit_price"] = dataf["maximum_unit_price"].where(~replace_filter_max, replace_series_max)

    dataf["minimum_unit_price"] = dataf["minimum_unit_price"].str.replace("$", "")
    dataf["maximum_unit_price"] = dataf["maximum_unit_price"].str.replace("$", "")

    return dataf


def clean_unit_of_measure(dataf):
    map_dict = {"bolsas": "bags",
                "bolosas": "bags",
                "docenas": "dozens"}

    dataf["unit_of_measure"] = dataf["unit_of_measure"].replace(map_dict)

    return dataf

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = ((isnull_sum >= 5)
                   | ((isnull_sum == 4) & (dataf["has_sold"] == "no")))
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf


def clean_values(dataf):
    dataf = clean_date_range(dataf)
    dataf = clean_has_sold(dataf)
    dataf = clean_commercial_fraction(dataf)
    dataf = clean_quantity_sold(dataf)
    dataf = clean_unit_price(dataf)
    dataf = clean_unit_of_measure(dataf)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, product_list,
                       translation_list, column_headers)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(clean_values)
            .pipe(handle_null_data))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame
Create the `dropped_df` and collect the rows which contain no non-null values

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

# get rows that were dropped but do not contain information that
# was captured correctly
null_df = dropped_df.loc[(~dropped_df["has_sold"].isin(["no", "n"]))
                         & (~dropped_df["product"].isin(["1", "2", "3", "4"])),
                         ["filename", "product"]]

### Check Dropped Row

In [ ]:
def get_dropped_row(section_search, product,
                    product_list, translation_list):
    dropped_row = {}

    if len(section_search) == 1:
        section_string = section_search[0] + "v-3:"

        dropped_row["product"] = product

        product_index = translation_list.index(product)
        item_1 = product_list[product_index]
        item_2 = product_list[product_index + 1]

        row_search = find_between(item_1, item_2, section_string)

        # if product row found
        if len(row_search) == 1:
            row_string = row_search[0]
            row_list = row_string.split("\n")
            row_list_len = len(row_list)

            if row_list_len >= 13:
                count = -2
                for j in range(row_list_len):
                    if count > 8:
                        break
                    if count >= 0:
                        col = str(count)
                        entry = row_list[j]
                        entry = entry.strip()
                        dropped_row[col] = entry
                    count += 1

            else:
                for j in range(9):
                    dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
        else:
            for j in range(9):
                dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
    else:
        dropped_row = {}

    return dropped_row

### Check Dropped Rows for Parsing Errors

In [ ]:
def check_dropped_rows(source_directory, product_list, translation_list, s):
    process_file = s["filename"]
    product = s["product"]

    check_dict = {}

    file_path = os.path.join(source_directory, process_file)

    # with statements automatically control the closing of files
    with open(file_path, "r") as file:
        contents = file.read()
        contents = contents.lower()

        section_start = "v-3: ventas de productos"
        section_end = "v-4: otros ingresos"
        section_search = find_between(section_start, section_end, contents)

        dropped_row = get_dropped_row(section_search, product,
                                      product_list, translation_list)

        # check to make sure files contain the right amount of entries
        if len(dropped_row) <= 1:
            print(f"Unencountered Format: {process_file}\n"
                  + f"Length of data: {len(dropped_row)}\n")

        check_dict["filename"] = process_file

        for key, value in dropped_row.items():
            check_dict[key] = value

    # convert list to Series
    check_series = pd.Series(data=check_dict)

    return check_series

In [ ]:
check_df = null_df.apply(lambda s: check_dropped_rows(source_directory,
                                                      product_list,
                                                      translation_list,
                                                      s),
                         axis=1)

In [ ]:
def drop_null_check(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 9
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf

In [ ]:
non_null_df = (check_df
               .pipe(start_pipeline)
               .pipe(drop_null_check))

In [ ]:
non_null_df

## Load
### Export to CSV

In [ ]:
if non_null_df.shape[0] == 0:
    datasets_directory = "../datasets/"
    filename = "section_5_3.csv"
    file_path = os.path.join(datasets_directory, filename)

    clean_df.to_csv(file_path, index=False, na_rep="null")

    print(f"Exported to {filename}")
else:
    print("Error: Dropped rows contain non-null values")

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]